# Full breakdown for the creation of the possiblePrograms.csv element spreadsheet that's necessary for dropdown menus

Import statements necessary to run program. If any of them aren't present, use the \<library\> name in the error message and put in "pip install \<library\>" or look up installation instructions on installing that specific library.

In [32]:
import pandas as pd
import os
import geocoder
import numpy as np
import csv
import re
# url = 'https://nominatim.openstreetmap.org/search?country=DE&city=Erlangen&postalcode=91052&street=N%C3%BCrnberger+Stra%C3%9Fe+7&format=json&limit=1'

# resp = http.request('GET', url)

Here we use the file that has been most recently populated which contains all the institutions (naming scheme outdated). At the time of this program's writing, that was a copied file called "simplifiedSchools(4-17-24).csv"

In [25]:
fileLoc = os.getcwd()+"\\simplifiedSchools(4-17-24).csv"

He're, we're opening the file at its above location and using a specific encoding relevant to the CSV file that we selected. All entries are of the string datatype, and we want to indicate that the header is the first row of the document.

In [26]:
df = pd.read_csv(fileLoc,header=0,dtype=str,encoding='cp1252')

Minor thing here to replace spaces with underscores for easier interacations with the code itself (less issues that way)

In [27]:
df.columns = [c.replace(' ', '_') for c in df.columns]

Here, we establish the different institution types that have associated programs. Not all institutions have special programs for each school, hence why they aren't listed.

In [28]:
types = ["High School","CTE","Community College","University/Colleges","Graduate","Company"]

Once we have the institution types, we go through the entire csv in the form of a dataframe and look at each type individually. We create a full list of programs for each type in this section and then sort those lists after making sure all elements in them are unique.

In [40]:
dictionary = {}
for institutionType in types:
    if institutionType != "Company":
        institution = df[(df["Institution_Type"] == institutionType) & ~(df["Programs"].isnull())]["Programs"].values
        newInstitution = []
        for i, element in enumerate(institution):
            elementList = []
            for newThing in element.split(","):
                if(newThing != ""):
                    elementList.append(newThing)
            
            newInstitution.append(elementList)
        institution = newInstitution
        newInstitution = []
        for i, element in enumerate(institution):
            newInstitution = np.concatenate((newInstitution,element))
        for i, element in enumerate(newInstitution):
            newInstitution[i] = element.strip()
        institPrograms = sorted(list(set(newInstitution)))
        dictionary[institutionType] = institPrograms
    else:
        institution = df[(df["Institution_Type"] == institutionType) & ~(df["Programs"].isnull())]["Programs"].values
        elementList = []
        for i, element in enumerate(institution):
            newElement = re.findall( r'[0-9]{6}', element)
            for newThing in newElement:
                if(newThing != ""):
                    elementList.append(newThing)
        institution = elementList
        institPrograms = sorted(list(set(institution)))
        dictionary[institutionType] = institPrograms

Everything is in a dictionary with its associated institution type, but in order to be properly formatted for Javascript, it needs a header, so we create a new dictionary with two separate keys, one for types and one for programs.

In [42]:
dictionary = {"types" : dictionary.keys(), "programs" : dictionary.values()}

We now create a new dataframe from this dictionary so we can export it, we specify index "false" so it doesn't include an additional column for unnecessary indices associated with each of the type/program combinations. It doesn't hurt to have it, but its not necessary.

In [43]:
dataframe = pd.DataFrame.from_dict(dictionary)
dataframe.to_csv(os.getcwd()+"\\possiblePrograms.csv", index=False)